# Multi Agent training using Malmo
This example expands on the single agent training example and shows how to use a multi-agent environment using RLlib. Some concepts are explained in more details in the previous examples.

Before we look at the code, let's go through how the multi-agent differs from the single-agent version.

Each agent requires a new Malmo instance. The connection is similar to the single-agent example, the main difference is that the first instance is going to act as a server and the other instances join as clients. The figure below shows a 2 agent example in which ```instance 0``` acts as the server and ```instance 1``` is the client. The agents only communicate with their instances and the instances communicate with each other as the arrows show.
Agent 1 sends actions to instance 1 which forwards it to instance 0. Then Minecraft updates and returns the new observation to instance 1 which forwards it to agent 1.

![Multi-agent setup](../imgs/malmo_multiagent_wrapper.png "Multi-agent wrapper")

In [ ]:
# imports
from pathlib import Path
import os
import gym

# malmoenv imports
import malmoenv
from malmoenv.utils.launcher import launch_minecraft
from malmoenv.utils.wrappers import DownsampleObs
from malmoenv.turnbasedmultiagentenv import AgentConfig, TurnBasedRllibMultiAgentEnv

import ray
from ray.tune import register_env

The next step is to define some constants. These are mostly the same as for the single-agent example.
The main difference here is that ```NUM_WORKERS``` here represents the number of malmo environments and not the number of agents. Care should be taken when assigning resources as using this example uses 2 CPU cores per env.

In [ ]:
MULTI_AGENT_ENV = "malmo_multi_agent"
MISSION_XML = os.path.realpath('../../MalmoEnv/missions/mobchase_single_agent.xml')
COMMAND_PORT = 8999 # first port's number
xml = Path(MISSION_XML).read_text()

CHECKPOINT_FREQ = 100      # in terms of number of algorithm iterations
LOG_DIR = "results/"       # creates a new directory and puts results there

NUM_WORKERS = 1            # number of environments to run - each env get multiple agents
NUM_GPUS = 0
TOTAL_STEPS = int(1e6)
launch_script = "./launchClient_quiet.sh"

Next we want to create a function that defines how the environment is generated in RLlib. This is going to be the python client connecting to the malmo instances, so make sure that these PORT numbers match the ports used later to create the Minecraft instances.
When using RLlib each worker has an index accessible by calling ```config.worker_index```, using this variable we can easily set the correct ports for each env.
If we would like to use wrappers the ```env_factory``` function is a good place to add them, see the ```DownsampleObs``` wrapper added in this example.
To use RLlib we have created 2 functions:
- ```env_factory```: Starts up a Malmo instance as we did with the ```create_env``` function in the Single agent example
- ```create_multi_agent_env```: Assigns the correct roles to the agents and wrap the environments using the TurnBasedRllibMultiAgentEnv.

Finally we have to register the env generator function to make it visible to RLlib.

In [ ]:
def env_factory(agent_id, xml, role, host_address, host_port, command_address, command_port):
    env = malmoenv.make()
    env.init(xml, host_port,
             server=host_address,
             server2=command_address,
             port2=command_port,
             role=role,
             exp_uid="multiagent",
             reshape=True
             )
    env = DownsampleObs(env, shape=(84, 84))

    return env

def create_multi_agent_env(config):
    port = COMMAND_PORT + (config.worker_index * 2)
    agent_config = [
        AgentConfig(id=f"agent1", address=port),
        AgentConfig(id=f"agent2", address=port + 1),
    ]
    env = TurnBasedRllibMultiAgentEnv(xml, agent_config,
                                      env_factory=env_factory,)
    return env

register_env(MULTI_AGENT_ENV, create_multi_agent_env)

The next step is to start up the Minecraft instances. Note that this step might take a few minutes.
In the background each Malmo instance get copied to the ```/tmp/malmo_<hash>/malmo``` directory, where it gets executed (Each Minecraft instance requires its own directory).
After copying the instances are started using a the provided ```launch_script```, this is where we can define if we want to run it without rendering a window for example.
By default it uses the ```launchClient_quiet.sh``` script which renders each window, so after executing the next cell you should see 2 windows rendered on your screen. It might take a few minutes depending on your hardware.

In [ ]:
GAME_INSTANCE_PORTS = [COMMAND_PORT + 1 + i for i in range(NUM_WORKERS)]
instances = launch_minecraft(GAME_INSTANCE_PORTS, launch_script=launch_script)

The multi-agent setup requires an extra argument: ```multiagent```, which defines how to handle the separate policies.
For more information on multi-agent environments read the "MultiAgentEnv" section in the [RLlib documentation](https://docs.ray.io/en/master/rllib-env.html)

In [ ]:
ray.tune.run(
    "PPO",
    config={
        "env": MULTI_AGENT_ENV,
        "num_workers": NUM_WORKERS,
        "num_gpus": NUM_GPUS,
        "multiagent": {
            "policies": { "shared_policy": (
                None,
                gym.spaces.Box(0, 255, shape=(84, 84, 3)),
                gym.spaces.Discrete(5),
                {}
            )},
            "policy_mapping_fn": (lambda agent_id: "shared_policy")
        }
    },
    stop={"timesteps_total": TOTAL_STEPS},
    checkpoint_at_end=True,
    checkpoint_freq=CHECKPOINT_FREQ,
    local_dir=LOG_DIR
)